# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules

*Выбор данных*

In [55]:
rawData = pd.read_csv("lastfm.csv")
data = rawData[rawData.country.isin([ 'Spain', 'Portugal', 'France', 'Italy', 'Belgium'])]

In [7]:
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


**2. Поиск ассоциативных правил**

Предобработка данных

In [9]:
data[data.duplicated(keep=False)]

,user,artist,sex,country


Дубликатов нет

In [10]:
data.isnull().sum()


user       0
artist     0
sex        0
country    0
dtype: int64

In [56]:
data[(data['artist'] == '[unknown]')]

,user,artist,sex,country
3540,242,[unknown],m,France
5763,394,[unknown],m,Belgium
8403,560,[unknown],f,Italy
19811,1302,[unknown],m,Spain
21071,1394,[unknown],m,Spain
27559,1826,[unknown],m,Italy
30270,2045,[unknown],m,Spain
41623,2830,[unknown],m,France
42097,2865,[unknown],f,Belgium
42542,2900,[unknown],m,Italy


In [57]:
data = data[(data['artist'] != '[unknown]')]

Формирование датафреймов для обучения модели

In [35]:
data_by_artists = data.groupby(['user', 'sex', 'country'])['artist'].apply(','.join).reset_index()
data_by_artists_dummies = data_by_artists['artist'].str.get_dummies(',')
data_by_artists_country_dummies = pd.concat([data_by_artists['country'].str.get_dummies(','),
                             data_by_artists['artist'].str.get_dummies(',')], axis = 1)

Поиск частых наборов алгоритмом Apriori

In [36]:
frequent_itemsets = apriori(data_by_artists_dummies, min_support=0.02, use_colnames=True)

In [37]:
frequent_itemsets.sort_values(by=['support'],ascending=False).head(1)

,support,itemsets
208,0.237826,(radiohead)


Выделание правил

In [38]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)
rulesb

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(muse),0.043754,0.174312,0.021171,0.483871,2.775891,0.013545,1.599771
1,(ac/dc),(iron maiden),0.074100,0.073394,0.021877,0.295238,4.022619,0.016439,1.314778
2,(iron maiden),(ac/dc),0.073394,0.074100,0.021877,0.298077,4.022619,0.016439,1.319090
3,(led zeppelin),(ac/dc),0.098800,0.074100,0.023289,0.235714,3.181020,0.015968,1.211458
4,(ac/dc),(led zeppelin),0.074100,0.098800,0.023289,0.314286,3.181020,0.015968,1.314250
...,...,...,...,...,...,...,...,...,...
755,(the killers),"(the strokes, muse)",0.110092,0.038109,0.024700,0.224359,5.887346,0.020505,1.240124
756,"(the beatles, radiohead)",(pink floyd),0.070572,0.130558,0.026112,0.370000,2.834000,0.016898,1.380067
757,"(the beatles, pink floyd)",(radiohead),0.048694,0.237826,0.026112,0.536232,2.254720,0.014531,1.643437
758,"(pink floyd, radiohead)",(the beatles),0.055046,0.174312,0.026112,0.474359,2.721323,0.016516,1.570821


**Правило 1**

Если правило имеет лифт 1, это означает, что событие в левой части независимо от события в правой части. Если два события независимы, никакого правила нельзя вытащить из этих двух событий.

Если лифт > 1, это позволяет нам знать степень, насколько события связаны друг с другом, и делает эти правила потенциально полезными для предсказания следствия в будущих наборах данных.

Если лифт < 1, это означает, что объекты заменяют друг друга. Это означает, что наличие одного объекта имеет отрицательный эффект на присутствие второго объекта, и наоборот. 

Так можно сделать вывод что люди слушающие ac/dc будут слушать iron maiden, metallica, led zeppelin и, на удивление, beatles 

Но при этом исходя из confidence, мы не можем рекомендовать слушателем metallica группу ac/dc с такой же уверенностью

In [67]:
rule1 = rules[(rules['antecedents'] == {'ac/dc'}) | (rules['consequents'] == {'ac/dc'} ) 
              ].sort_values(by = ['lift'], ascending=False)
rule1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(ac/dc),(iron maiden),0.074100,0.073394,0.021877,0.295238,4.022619,0.016439,1.314778
2,(iron maiden),(ac/dc),0.073394,0.074100,0.021877,0.298077,4.022619,0.016439,1.319090
5,(ac/dc),(metallica),0.074100,0.123500,0.029640,0.400000,3.238857,0.020489,1.460833
6,(metallica),(ac/dc),0.123500,0.074100,0.029640,0.240000,3.238857,0.020489,1.218289
3,(led zeppelin),(ac/dc),0.098800,0.074100,0.023289,0.235714,3.181020,0.015968,1.211458
4,(ac/dc),(led zeppelin),0.074100,0.098800,0.023289,0.314286,3.181020,0.015968,1.314250
7,(ac/dc),(the beatles),0.074100,0.174312,0.020466,0.276190,1.584461,0.007549,1.140753


**Правило 2**

In [69]:
rule1.sort_values(by = ['conviction'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(ac/dc),(metallica),0.074100,0.123500,0.029640,0.400000,3.238857,0.020489,1.460833
2,(iron maiden),(ac/dc),0.073394,0.074100,0.021877,0.298077,4.022619,0.016439,1.319090
1,(ac/dc),(iron maiden),0.074100,0.073394,0.021877,0.295238,4.022619,0.016439,1.314778
4,(ac/dc),(led zeppelin),0.074100,0.098800,0.023289,0.314286,3.181020,0.015968,1.314250
6,(metallica),(ac/dc),0.123500,0.074100,0.029640,0.240000,3.238857,0.020489,1.218289
3,(led zeppelin),(ac/dc),0.098800,0.074100,0.023289,0.235714,3.181020,0.015968,1.211458
7,(ac/dc),(the beatles),0.074100,0.174312,0.020466,0.276190,1.584461,0.007549,1.140753


convinction равное величине x показывает, что правило будет неверным в x раз чаще, если ассоциация между X и Y была чистой случайностью.

Чем ближе convinction к 1 тем точнее правило.

Ассоциация ac/dc -> beatles будет неверным на 14% чаще, если ассоциация между ними была чистой случайностью, т.е. если это правило некорректно, то степень ошибки увеличивается несильно

**Правило 3**

In [73]:
frequent_itemsets_fpgrowth = fpgrowth(data_by_artists_dummies, min_support=0.01, use_colnames=True)

In [74]:
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.5)
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(muse, coldplay)",(radiohead),0.068454,0.237826,0.038814,0.567010,2.384135,0.022534,1.760258
1,"(the beatles, muse)",(radiohead),0.036697,0.237826,0.025406,0.692308,2.910979,0.016678,2.477064
2,"(the beatles, muse, coldplay)",(radiohead),0.016937,0.237826,0.011291,0.666667,2.803165,0.007263,2.286521
3,"(muse, pink floyd)",(radiohead),0.033169,0.237826,0.020466,0.617021,2.594419,0.012577,1.990120
4,"(muse, pink floyd)",(the beatles),0.033169,0.174312,0.016937,0.510638,2.929451,0.011155,1.687276
...,...,...,...,...,...,...,...,...,...
2845,"(stereophonics, radiohead)",(coldplay),0.012703,0.188426,0.011291,0.888889,4.717437,0.008898,7.304164
2846,(stereophonics),"(radiohead, coldplay)",0.021171,0.083275,0.011291,0.533333,6.404520,0.009528,1.964412
2847,"(oasis, stereophonics)",(coldplay),0.011997,0.188426,0.010586,0.882353,4.682750,0.008325,6.898377
2848,"(stereophonics, coldplay)",(oasis),0.016231,0.083275,0.010586,0.652174,7.831614,0.009234,2.635586


In [76]:
rules_fpgrowth["antecedent_len"] = rules_fpgrowth["antecedents"].apply(lambda x: len(x))
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,"(muse, coldplay)",(radiohead),0.068454,0.237826,0.038814,0.567010,2.384135,0.022534,1.760258,2
1,"(the beatles, muse)",(radiohead),0.036697,0.237826,0.025406,0.692308,2.910979,0.016678,2.477064,2
2,"(the beatles, muse, coldplay)",(radiohead),0.016937,0.237826,0.011291,0.666667,2.803165,0.007263,2.286521,3
3,"(muse, pink floyd)",(radiohead),0.033169,0.237826,0.020466,0.617021,2.594419,0.012577,1.990120,2
4,"(muse, pink floyd)",(the beatles),0.033169,0.174312,0.016937,0.510638,2.929451,0.011155,1.687276,2
...,...,...,...,...,...,...,...,...,...,...
2845,"(stereophonics, radiohead)",(coldplay),0.012703,0.188426,0.011291,0.888889,4.717437,0.008898,7.304164,2
2846,(stereophonics),"(radiohead, coldplay)",0.021171,0.083275,0.011291,0.533333,6.404520,0.009528,1.964412,1
2847,"(oasis, stereophonics)",(coldplay),0.011997,0.188426,0.010586,0.882353,4.682750,0.008325,6.898377,2
2848,"(stereophonics, coldplay)",(oasis),0.016231,0.083275,0.010586,0.652174,7.831614,0.009234,2.635586,2


In [79]:
rule3 = rules_fpgrowth[(rules_fpgrowth['antecedent_len'] > 1) ].sort_values(by=["antecedent_len"])
rule3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,"(muse, coldplay)",(radiohead),0.068454,0.237826,0.038814,0.567010,2.384135,0.022534,1.760258,2
1382,"(justice, digitalism)",(daft punk),0.013409,0.102329,0.010586,0.789474,7.715064,0.009214,4.263938,2
1381,"(daft punk, digitalism)",(justice),0.016937,0.038109,0.010586,0.625000,16.400463,0.009940,2.565044,2
1379,"(the hives, the killers)",(interpol),0.016937,0.079746,0.010586,0.625000,7.837389,0.009235,2.454011,2
1378,"(interpol, the hives)",(the killers),0.014820,0.110092,0.010586,0.714286,6.488095,0.008954,3.114679,2
...,...,...,...,...,...,...,...,...,...,...
603,"(oasis, muse, the strokes, the killers)",(bloc party),0.013409,0.083275,0.010586,0.789474,9.480375,0.009469,4.354446,4
604,"(oasis, bloc party, muse, the killers)",(the strokes),0.013409,0.087509,0.010586,0.789474,9.021647,0.009412,4.334333,4
605,"(the strokes, bloc party, muse, the killers)",(oasis),0.015526,0.083275,0.010586,0.681818,8.187596,0.009293,2.881137,4
1807,"(arctic monkeys, franz ferdinand, the killers,...",(muse),0.011997,0.174312,0.010586,0.882353,5.061920,0.008494,7.018349,4


In [90]:
rule3 = rules_fpgrowth[(rules_fpgrowth['antecedent_len'] > 1)]
rule3 = rule3.sort_values(by=["support"],ascending=False)

In [92]:
rule3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,"(muse, coldplay)",(radiohead),0.068454,0.237826,0.038814,0.567010,2.384135,0.022534,1.760258,2
873,"(muse, the killers)",(coldplay),0.049400,0.188426,0.029640,0.600000,3.184270,0.020332,2.028934,2
874,"(the killers, coldplay)",(muse),0.055752,0.174312,0.029640,0.531646,3.049967,0.019922,1.762956,2
2367,"(interpol, radiohead)",(muse),0.045872,0.174312,0.028229,0.615385,3.530364,0.020233,2.146789,2
2366,"(interpol, muse)",(radiohead),0.040932,0.237826,0.028229,0.689655,2.899826,0.018494,2.455893,2
...,...,...,...,...,...,...,...,...,...,...
1755,"(franz ferdinand, daft punk)",(coldplay),0.020466,0.188426,0.010586,0.517241,2.745060,0.006729,1.681117,2
363,"(the strokes, the cure)",(muse),0.021171,0.174312,0.010586,0.500000,2.868421,0.006895,1.651376,2
359,"(arctic monkeys, the killers, radiohead)",(the strokes),0.015526,0.087509,0.010586,0.681818,7.791422,0.009227,2.867829,3
1767,"(franz ferdinand, muse, placebo)",(coldplay),0.013409,0.188426,0.010586,0.789474,4.189829,0.008059,3.854975,3


In [107]:
rule3.groupby(['antecedents']).count().sort_values(by=["antecedent_len"],ascending=False)["antecedent_len"]

antecedents
(oasis, bloc party)                    20
(the strokes, bloc party, coldplay)    19
(oasis, interpol)                      19
(franz ferdinand, editors)             16
(the strokes, arctic monkeys, muse)    16
                                       ..
(arctic monkeys, maxïmo park)           1
(franz ferdinand, maxïmo park)          1
(muse, the chemical brothers)           1
(bob dylan, led zeppelin)               1
(franz ferdinand, muse, placebo)        1
Name: antecedent_len, Length: 857, dtype: int64

In [111]:
rule3[rule3["antecedents"] == {"oasis", "bloc party"}].sort_values(by=["confidence","support"],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
531,"(oasis, bloc party)",(the killers),0.022583,0.110092,0.017643,0.78125,7.096354,0.015157,4.068152,2
534,"(oasis, bloc party)",(coldplay),0.022583,0.188426,0.017643,0.78125,4.146184,0.013388,3.710051,2
535,"(oasis, bloc party)",(muse),0.022583,0.174312,0.016231,0.71875,4.123355,0.012295,2.935780,2
544,"(oasis, bloc party)","(coldplay, the killers)",0.022583,0.055752,0.015526,0.68750,12.331487,0.014267,3.021595,2
549,"(oasis, bloc party)","(muse, coldplay)",0.022583,0.068454,0.014114,0.62500,9.130155,0.012568,2.484121,2
554,"(oasis, bloc party)","(muse, the killers)",0.022583,0.049400,0.013409,0.59375,12.019196,0.012293,2.339938,2
539,"(oasis, bloc party)",(arctic monkeys),0.022583,0.101623,0.013409,0.59375,5.842665,0.011114,2.211389,2
566,"(oasis, bloc party)","(muse, coldplay, the killers)",0.022583,0.029640,0.012703,0.56250,18.977679,0.012034,2.217966,2
537,"(oasis, bloc party)",(the strokes),0.022583,0.087509,0.012703,0.56250,6.427923,0.010727,2.085694,2
630,"(oasis, bloc party)","(arctic monkeys, muse)",0.022583,0.046577,0.012703,0.56250,12.076705,0.011651,2.179252,2


Слушатели которые предопчитают Oasis и bloc party имеют наибольшее кол-во групп которые им могут быть потенциально интересны. Однако отсортировав правила по confidence и support можно с большой уверенностью ожидать, что слушателям этих групп понравится the killers. Повышенное значение conviction связано с маленьким значением support  

**Правило 4**

In [167]:
frequent_itemsets_fpgrowth2 = fpgrowth(data_by_artists_country_dummies, min_support=0.01, use_colnames=True)

In [168]:
rules_fpgrowth2 = association_rules(frequent_itemsets_fpgrowth2, metric="lift", min_threshold=0.3)
rules_fpgrowth2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
1,(Spain),(muse),0.355681,0.174312,0.067749,0.190476,1.092732,0.005749,1.019968
2,(muse),(France),0.174312,0.230769,0.038814,0.222672,0.964912,-0.001411,0.989583
3,(France),(muse),0.230769,0.174312,0.038814,0.168196,0.964912,-0.001411,0.992647
4,(muse),(radiohead),0.174312,0.237826,0.080452,0.461538,1.940653,0.038996,1.415465
...,...,...,...,...,...,...,...,...,...
16513,"(oasis, coldplay)",(stereophonics),0.047989,0.021171,0.010586,0.220588,10.419118,0.009570,1.255855
16514,"(stereophonics, coldplay)",(oasis),0.016231,0.083275,0.010586,0.652174,7.831614,0.009234,2.635586
16515,(oasis),"(stereophonics, coldplay)",0.083275,0.016231,0.010586,0.127119,7.831614,0.009234,1.127036
16516,(stereophonics),"(oasis, coldplay)",0.021171,0.047989,0.010586,0.500000,10.419118,0.009570,1.904023


In [190]:
rule4 = rules_fpgrowth2[((rules_fpgrowth2['consequents'] == { 'Spain'}) | (rules_fpgrowth2['consequents'] == { 'Italy'}) | (rules_fpgrowth2['consequents'] == { 'Portugal'}) | (rules_fpgrowth2['consequents'] == { 'France'}) |(rules_fpgrowth2['consequents'] == { 'Belgium'}))]
rule4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
2,(muse),(France),0.174312,0.230769,0.038814,0.222672,0.964912,-0.001411,0.989583
8,(muse),(Italy),0.174312,0.214538,0.033874,0.194332,0.905817,-0.003522,0.974920
10,"(muse, coldplay)",(Spain),0.068454,0.355681,0.028229,0.412371,1.159385,0.003881,1.096473
16,"(muse, radiohead)",(Spain),0.080452,0.355681,0.023289,0.289474,0.813858,-0.005326,0.906819
...,...,...,...,...,...,...,...,...,...
16454,(the mars volta),(Spain),0.024700,0.355681,0.010586,0.428571,1.204932,0.001800,1.127558
16456,(various artists),(Spain),0.021171,0.355681,0.013409,0.633333,1.780622,0.005878,1.757234
16460,(fatboy slim),(Spain),0.033169,0.355681,0.011997,0.361702,1.016928,0.000200,1.009433
16480,(deftones),(France),0.035286,0.230769,0.010586,0.300000,1.300000,0.002443,1.098901


In [200]:
rule4 = rule4.sort_values(by=['lift'], ascending=False)
rule4.head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7829,(deus),(Belgium),0.040226,0.112915,0.021877,0.543860,4.816557,0.017335,1.944764
7842,"(radiohead, deus)",(Belgium),0.021877,0.112915,0.011291,0.516129,4.570968,0.008821,1.833310
16295,(hooverphonic),(Belgium),0.029640,0.112915,0.014114,0.476190,4.217262,0.010768,1.693527
7863,(eels),(Belgium),0.033874,0.112915,0.011997,0.354167,3.136589,0.008172,1.373552
16112,"(the clash, pink floyd)",(Italy),0.017643,0.214538,0.011291,0.640000,2.983158,0.007506,2.181840
5514,(serge gainsbourg),(France),0.026817,0.230769,0.017643,0.657895,2.850877,0.011454,2.248521
16310,"(mando diao, coldplay)",(Spain),0.010586,0.355681,0.010586,1.000000,2.811508,0.006821,inf
6868,"(r.e.m., the beatles)",(Italy),0.019054,0.214538,0.011291,0.592593,2.762183,0.007204,1.927953
16162,(archive),(France),0.024700,0.230769,0.015526,0.628571,2.723810,0.009826,2.071006
14998,(explosions in the sky),(Belgium),0.035286,0.112915,0.010586,0.300000,2.656875,0.006601,1.267265


In [192]:
rule4[rule4['antecedents'] == {'mando diao'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16298,(mando diao),(Spain),0.018349,0.355681,0.016231,0.884615,2.487103,0.009705,5.584098


Группа mando diao исключительно испанская группа и не будет пользоваться популярностью в других странах. 

**Правило 5**

In [193]:
rule5 = rules_fpgrowth2[((rules_fpgrowth2['consequents'] == { 'Spain'}) | (rules_fpgrowth2['consequents'] == { 'Italy'}) | (rules_fpgrowth2['consequents'] == { 'Portugal'}) | (rules_fpgrowth2['consequents'] == { 'France'}) |(rules_fpgrowth2['consequents'] == { 'Belgium'}))]
rule5

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
2,(muse),(France),0.174312,0.230769,0.038814,0.222672,0.964912,-0.001411,0.989583
8,(muse),(Italy),0.174312,0.214538,0.033874,0.194332,0.905817,-0.003522,0.974920
10,"(muse, coldplay)",(Spain),0.068454,0.355681,0.028229,0.412371,1.159385,0.003881,1.096473
16,"(muse, radiohead)",(Spain),0.080452,0.355681,0.023289,0.289474,0.813858,-0.005326,0.906819
...,...,...,...,...,...,...,...,...,...
16454,(the mars volta),(Spain),0.024700,0.355681,0.010586,0.428571,1.204932,0.001800,1.127558
16456,(various artists),(Spain),0.021171,0.355681,0.013409,0.633333,1.780622,0.005878,1.757234
16460,(fatboy slim),(Spain),0.033169,0.355681,0.011997,0.361702,1.016928,0.000200,1.009433
16480,(deftones),(France),0.035286,0.230769,0.010586,0.300000,1.300000,0.002443,1.098901


In [196]:
rule5 = rule5.sort_values(by=['antecedent support'], ascending=False)
rule5

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1123,(radiohead),(Italy),0.237826,0.214538,0.060692,0.255193,1.189501,0.009669,1.054585
649,(radiohead),(Portugal),0.237826,0.086097,0.024700,0.103858,1.206280,0.004224,1.019818
2082,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546
5625,(radiohead),(Belgium),0.237826,0.112915,0.027523,0.115727,1.024907,0.000669,1.003180
5457,(radiohead),(France),0.237826,0.230769,0.062103,0.261128,1.131553,0.007220,1.041087
...,...,...,...,...,...,...,...,...,...
14864,"(the offspring, the killers)",(Spain),0.014114,0.355681,0.011291,0.800000,2.249206,0.006271,3.221595
9562,"(franz ferdinand, the killers, foo fighters)",(Spain),0.013409,0.355681,0.010586,0.789474,2.219612,0.005817,3.060515
16316,"(mando diao, the killers)",(Spain),0.012703,0.355681,0.011291,0.888889,2.499118,0.006773,5.798871
16322,"(mando diao, muse)",(Spain),0.011291,0.355681,0.010586,0.937500,2.635789,0.006570,10.309104


Группа radiohead наиболее популярна среди остальных групп в данных странах. Определим в каких странах их гастроли буду пользоваться наибольшей популярностью

In [197]:
rule5[rule5['antecedents'] == {'radiohead'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1123,(radiohead),(Italy),0.237826,0.214538,0.060692,0.255193,1.189501,0.009669,1.054585
649,(radiohead),(Portugal),0.237826,0.086097,0.024700,0.103858,1.206280,0.004224,1.019818
2082,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546
5625,(radiohead),(Belgium),0.237826,0.112915,0.027523,0.115727,1.024907,0.000669,1.003180
5457,(radiohead),(France),0.237826,0.230769,0.062103,0.261128,1.131553,0.007220,1.041087


In [198]:
rule5[rule5['antecedents'] == {'radiohead'}].sort_values(by=["confidence","support"],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2082,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546
5457,(radiohead),(France),0.237826,0.230769,0.062103,0.261128,1.131553,0.007220,1.041087
1123,(radiohead),(Italy),0.237826,0.214538,0.060692,0.255193,1.189501,0.009669,1.054585
5625,(radiohead),(Belgium),0.237826,0.112915,0.027523,0.115727,1.024907,0.000669,1.003180
649,(radiohead),(Portugal),0.237826,0.086097,0.024700,0.103858,1.206280,0.004224,1.019818


Гастроли группы radiohead будут наиболее прибыльными в Испании ,Франции и Италии?